Steps:
1) read in file
2) custom stop list
2.1) prefix words from pubmed ['conclusions', 'introduction', 'methods', 'purpose', 'results']
2.2) remove words by frequency (not apply yet, still looking for abnormality)
3) steming & remove stop words, remove numberic, remove puntuation
4) Apply POS

In [29]:
#1) get .csv directly from downloaded file
import os
import pandas as pd
path = os.getcwd()
pardir_path = os.path.abspath(os.path.join(path, os.pardir))
raw_name = 'news_data_9600'
input_folder_ = 'tina_test'
output_folder_ = 'text_mine_output'
format_ = '.csv'
raw_file_path = os.path.join(pardir_path, input_folder_,(raw_name+format_))

In [30]:
import pandas as pd

df = pd.read_csv(raw_file_path)

In [31]:
print(df.head(5))
print (f'raw data shape is {df.shape}') #(9600, 5)
print(df.columns) # ['Unnamed: 0', 'Title', 'Source', 'Published', 'Summary']

   Unnamed: 0                                              Title  \
0           1  Construction safety blitz on both sides of the...   
1           2  CBD apartment construction shut down over 'min...   
2           3  Construction of continuing care facility halte...   
3           4  Frightening drop in construction safety inspec...   
4           5  How inspections can improve construction site ...   

                                              Source  \
0  {'href': 'https://www.safework.nsw.gov.au', 't...   
1  {'href': 'https://www.illawarramercury.com.au'...   
2  {'href': 'https://www.thesafetymag.com', 'titl...   
3  {'href': 'https://www.unitetheunion.org', 'tit...   
4  {'href': 'https://www.thesafetymag.com', 'titl...   

                       Published  \
0  Tue, 16 Aug 2022 07:00:00 GMT   
1  Tue, 30 Aug 2022 07:59:15 GMT   
2  Tue, 30 Aug 2022 16:37:21 GMT   
3  Thu, 04 Aug 2022 07:00:00 GMT   
4  Wed, 03 Aug 2022 07:00:00 GMT   

                                     

In [32]:
# 1.1) add index record number
import numpy as np
df['rec_id'] = np.arange(1, df.shape[0] + 1)

In [35]:
# 1.2) convert time zone from GMT to sydney GMT+10
df['Published']  = pd.to_datetime(df['Published']).dt.tz_convert('Australia/Sydney')
print (df['Published'])

0      2022-08-16 17:00:00+10:00
1      2022-08-30 17:59:15+10:00
2      2022-08-31 02:37:21+10:00
3      2022-08-04 17:00:00+10:00
4      2022-08-03 17:00:00+10:00
                  ...           
9595   2022-03-16 18:00:00+11:00
9596   2022-08-24 20:16:54+10:00
9597   2022-08-31 12:59:07+10:00
9598   2022-08-28 01:16:00+10:00
9599   2022-08-30 20:03:17+10:00
Name: Published, Length: 9600, dtype: datetime64[ns, Australia/Sydney]


In [36]:
df.head()

,Unnamed: 0,Title,Source,Published,Summary,rec_id
0,1,Construction safety blitz on both sides of the...,"{'href': 'https://www.safework.nsw.gov.au', 't...",2022-08-16 17:00:00+10:00,Construction safety blitz on both sides of the...,1
1,2,CBD apartment construction shut down over 'min...,{'href': 'https://www.illawarramercury.com.au'...,2022-08-30 17:59:15+10:00,CBD apartment construction shut down over 'min...,2
2,3,Construction of continuing care facility halte...,"{'href': 'https://www.thesafetymag.com', 'titl...",2022-08-31 02:37:21+10:00,Construction of continuing care facility halte...,3
3,4,Frightening drop in construction safety inspec...,"{'href': 'https://www.unitetheunion.org', 'tit...",2022-08-04 17:00:00+10:00,Frightening drop in construction safety inspec...,4
4,5,How inspections can improve construction site ...,"{'href': 'https://www.thesafetymag.com', 'titl...",2022-08-03 17:00:00+10:00,How inspections can improve construction site ...,5


In [41]:
# 1.3) rename columns
df.rename(columns={ 'Title':'title', 'Source':'source','Published': 'published_date', 'Summary':'abstract'}, inplace=True)
print(f'Standardised column names : {df.columns}')

Standardised column names : Index(['Unnamed: 0', 'title', 'source', 'published_date', 'abstract',
       'rec_id'],
      dtype='object')


In [44]:
## 2. Create stopword list
#2) Prep file
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4') # use in stemming

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TinaM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\TinaM\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TinaM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\TinaM\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\TinaM\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [45]:
# 2.1) view the default stopword list:
stops = set(stopwords.words('english'))

pub_med_cust_stop = ['conclusions:', 'introduction:', 'methods:', 'purpose:', 'results:', 'objective:','background:']

stops.update(['br', 'href'],pub_med_cust_stop)

print(f'There are {len(stops)} words in the default stop words list.')
print(stops)

There are 188 words in the default stop words list.
{'had', "didn't", 'this', 'm', 'how', 'both', 'won', 'will', 'why', 'has', 'them', 't', 'she', 'over', 'again', 'objective:', 'what', 'was', 'hasn', 'my', 'themselves', "should've", 'conclusions:', 'is', 'our', 'been', "it's", 'didn', 'they', "haven't", 'o', 'mightn', 'd', 'a', 'hers', 've', 'where', "wasn't", 'i', 'other', 'some', 'about', 'yours', 'no', "mustn't", 'am', "hasn't", 'your', 'we', 'shouldn', 'those', 'in', 'who', 's', 'very', 'wouldn', 'him', 'there', 'which', 'but', 'needn', 'while', 'introduction:', 'ma', 'and', 'below', "hadn't", 'as', 'couldn', 'own', 'or', 'll', 'herself', 'me', 'most', 'background:', 'at', 'from', 'few', 'do', 'once', 'their', "mightn't", 'being', 'same', 'now', 'under', "you'll", "aren't", "shouldn't", 'whom', 'doing', 'his', 'haven', 'isn', 'were', 'purpose:', "that'll", "isn't", 'if', 'himself', 'yourself', 'such', 'theirs', 'for', 'an', "you're", "you've", "shan't", "wouldn't", 'he', 'because'

In [46]:
# 2.2) Create another customer stopword list from the word frequency
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [48]:
df['abstract']

0       Construction safety blitz on both sides of the...
1       CBD apartment construction shut down over 'min...
2       Construction of continuing care facility halte...
3       Frightening drop in construction safety inspec...
4       How inspections can improve construction site ...
                              ...                        
9595    Joint safety blitz aims to cut falls in constr...
9596    How Tideway is raising the bar for safety trai...
9597    Broughton Street business owners discuss impac...
9598    Nearly $51 mil awarded to county jail renovati...
9599    This much I know: 'I'm determined to break dow...
Name: abstract, Length: 9600, dtype: object

In [77]:
## remove stopwords from the abstract to see find the Tfidf
custom_abstract = df['abstract'].str.replace(r'[0-9]+', '')
custom_abstract =custom_abstract.apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stops))

print('Abstract after remvoing numeric and default stopwords')
custom_abstract 

C:\Users\TinaM\AppData\Local\Temp/ipykernel_21244/1362433962.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  custom_abstract = df['abstract'].str.replace(r'[0-9]+', '').apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stops))


Abstract after remvoing numeric and default stopwords


0                                            sides border
1                  cbd shut 'minefield' illawarra mercury
2                                         care halted due
3                    frightening drop exposed unite union
4                                                        
                              ...                        
9595                                            joint cut
9596                                          tideway bar
9597    broughton owners discuss impact working foxmed...
9598                       nearly $ mil awarded jail wtvg
9599           much know: 'i'm determined break barriers'
Name: abstract, Length: 9600, dtype: object

In [85]:
## remove stopwords from the title to see find the Tfidf
custom_title= df['title'].str.replace(r'[0-9]+', '')

print('Abstract after remvoing numeric and default stopwords')
custom_title

Abstract after remvoing numeric and default stopwords


C:\Users\TinaM\AppData\Local\Temp/ipykernel_21244/3491161362.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  custom_title= df['title'].str.replace(r'[0-9]+', '')


0       Construction safety blitz on both sides of the...
1       CBD apartment construction shut down over 'min...
2       Construction of continuing care facility halte...
3       Frightening drop in construction safety inspec...
4       How inspections can improve construction site ...
                              ...                        
9595    Joint safety blitz aims to cut falls in constr...
9596    How Tideway is raising the bar for safety trai...
9597    Broughton Street business owners discuss impac...
9598    Nearly $ mil awarded to county jail renovation...
9599    This much I know: 'I'm determined to break dow...
Name: title, Length: 9600, dtype: object

In [61]:
## initiate CountVectorizer
uni_count = CountVectorizer(stop_words=list(stops) + pub_med_cust_stop
                           ,ngram_range=(1,1) 
                           )

uni_word_count=uni_count.fit_transform(custom_abstract)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(uni_word_count)
## keep words in pub_med_cust_stop list if it's not follow by ':', as they might be part of the real text

### idf value
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=uni_count.get_feature_names(),columns=["idf_weights"])

print(f'IDF table - Abstract')
print(df_idf.sort_values(by=['idf_weights']))

print('Distribution of the idf_weights')
df_idf.idf_weights.describe()
## the lowest weight is 'cont'

IDF table.
              idf_weights
construction     1.437479
safety           1.506426
magazine         3.260868
health           3.273940
new              3.321617
...                   ...
says             9.071010
daily            9.071010
urgent           9.071010
opseu            9.071010
commercial       9.071010

[661 rows x 1 columns]
Distribution of the idf_weights


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['background', 'conclusions', 'introduction', 'methods', 'objective', 'purpose', 'results'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


count    661.000000
mean       5.556095
std        0.812163
min        1.437479
25%        5.554502
50%        5.554502
75%        5.554502
max        9.071010
Name: idf_weights, dtype: float64

In [70]:
## after removing the 
idf_stop = df_idf.idf_weights.describe()[5] ## idf_weights at 25%

print(f'words with idf_weights less than 25% across all docs, total {len(df_idf[df_idf.idf_weights<idf_stop])} words.')
print(df_idf[df_idf.idf_weights<idf_stop].sort_values(by=['idf_weights']))

idf_stop_list = list(df_idf[df_idf.idf_weights<idf_stop].sort_values(by=['idf_weights']).index)


words with idf_weights less than 25% across all docs, total 121 words.
              idf_weights
construction     1.437479
safety           1.506426
magazine         3.260868
health           3.273940
new              3.321617
...                   ...
protect          5.357438
help             5.433424
tech             5.525232
latest           5.534894
sun              5.534894

[121 rows x 1 columns]


In [83]:
## whether add to custom stop list
_to_add_stop_words = 'N'

if _to_add_stop_words == 'Y':
    stops_abs = list(stops) + list(idf_stop_list)
else:
    stops_abs = stops

print(f'There are {len(stops_abs)} final stop words for abstract.')
print(stops_abs)
## not to add low idf weigth words as want to keep 'construction', 'safety'

## output 1 A - stoplist for abstract
pd.DataFrame(list(stops_abs)).to_csv('final_stop_words_abstract.csv', index=True)

There are 309 final stop words for abstract.
['had', "didn't", 'this', 'm', 'how', 'both', 'won', 'will', 'why', 'has', 'them', 't', 'she', 'over', 'again', 'objective:', 'what', 'was', 'hasn', 'my', 'themselves', "should've", 'conclusions:', 'is', 'our', 'been', "it's", 'didn', 'they', "haven't", 'o', 'mightn', 'd', 'a', 'hers', 've', 'where', "wasn't", 'i', 'other', 'some', 'about', 'yours', 'no', "mustn't", 'am', "hasn't", 'your', 'we', 'shouldn', 'those', 'in', 'who', 's', 'very', 'wouldn', 'him', 'there', 'which', 'but', 'needn', 'while', 'introduction:', 'ma', 'and', 'below', "hadn't", 'as', 'couldn', 'own', 'or', 'll', 'herself', 'me', 'most', 'background:', 'at', 'from', 'few', 'do', 'once', 'their', "mightn't", 'being', 'same', 'now', 'under', "you'll", "aren't", "shouldn't", 'whom', 'doing', 'his', 'haven', 'isn', 'were', 'purpose:', "that'll", "isn't", 'if', 'himself', 'yourself', 'such', 'theirs', 'for', 'an', "you're", "you've", "shan't", "wouldn't", 'he', 'because', 'off'

In [86]:
#appy the same to title, initiate countervetorizer

title_uni_word_count=uni_count.fit_transform(custom_title)
title_tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
title_tfidf_transformer.fit(title_uni_word_count)
## keep words in pub_med_cust_stop list if it's not follow by ':', as they might be part of the real text

### idf value
title_df_idf = pd.DataFrame(title_tfidf_transformer.idf_, index=uni_count.get_feature_names(),columns=["idf_weights"])

print(f'IDF table - Title')
print(title_df_idf.sort_values(by=['idf_weights']))

print('Distribution of the idf_weights')
title_df_idf.idf_weights.describe()
## the lowest weight is 'cont'

IDF table - Title
              idf_weights
construction     1.437479
safety           1.506426
magazine         3.260868
health           3.273940
new              3.321617
...                   ...
says             9.071010
daily            9.071010
urgent           9.071010
opseu            9.071010
commercial       9.071010

[661 rows x 1 columns]
Distribution of the idf_weights


count    661.000000
mean       5.556095
std        0.812163
min        1.437479
25%        5.554502
50%        5.554502
75%        5.554502
max        9.071010
Name: idf_weights, dtype: float64

In [87]:
## after removing the 
title_idf_stop = title_df_idf.idf_weights.describe()[5] ## idf_weights at 25%

print(f'words with idf_weights less than 25% across all titles, total {len(title_df_idf[title_df_idf.idf_weights<idf_stop])} words.')
print(title_df_idf[title_df_idf.idf_weights<idf_stop].sort_values(by=['idf_weights']))

title_idf_stop_list = list(title_df_idf[title_df_idf.idf_weights<idf_stop].sort_values(by=['idf_weights']).index)

words with idf_weights less than 25% across all titles, total 121 words.
              idf_weights
construction     1.437479
safety           1.506426
magazine         3.260868
health           3.273940
new              3.321617
...                   ...
protect          5.357438
help             5.433424
tech             5.525232
latest           5.534894
sun              5.534894

[121 rows x 1 columns]


In [88]:
## whether add to custom stop list in title
_to_add_stop_words_title = 'N'

if _to_add_stop_words_title == 'Y':
    stops_title = list(idf_stop_list)
else:
    stops_title = ''

print(f'There are {len(stops_title)} final stop words for title.')
print(stops_title)

## output 1 B - stoplist for title
pd.DataFrame(list(stops_title)).to_csv('final_stop_words_title.csv', index=True)
## not to add low idf weigth words as want to keep 'construction', 'safety'

There are 0 final stop words for title.



In [89]:
# 3) Steming - abstract
stops = stops_abs
stemmer = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()

# POS (Parts Of Speech) for: NN = nouns, JJ = adjectives, VB= verbs and RB= adverbs, JJR = adjective, comparative (larger), WRB=wh- adverb (how), WP = wh- pronoun (who), WDT = wh-determiner (that, what)
DI_POS_TYPES = {'NN':'n', 'JJ':'a', 'VB':'v', 'RB':'r', 'JJR':'c', 'WRB':'b','WP':'p','WDT':'t'} 
POS_TYPES = list(DI_POS_TYPES.keys())

# Constraints on tokens
MIN_STR_LEN = 3
RE_VALID = '[a-zA-Z]' ## only keep alphabet

In [90]:
# convcert the text column to analysis to a list
abstracts= df.abstract.tolist()

print(f'Total of {len(abstracts)} entries.')

print('The first entry of abstracts')
print(abstracts[0])

print('The second entry of abstracts')
print(abstracts[1])

Total of 9600 entries.
The first entry of abstracts
Construction safety blitz on both sides of the border  SafeWork NSW
The second entry of abstracts
CBD apartment construction shut down over 'minefield' of safety issues  Illawarra Mercury


In [92]:
# 3.1) set up functions to remove accents
import unicodedata ## for remove accents function
import string ## for remove accents function
import re ## to remove character not in RE_VALID

In [93]:
## function from https://colab.research.google.com/github/gal-a/blog/blob/master/docs/notebooks/nlp/nltk_preprocess.ipynb#scrollTo=-44aMwUcQZxm
# Remove accents function
def remove_accents(data):
    return ''.join(x for x in unicodedata.normalize('NFKD', data) if x in string.ascii_letters or x == " ")

# Process all quotes
li_tokens = []
li_token_lists = []
li_lem_strings = []

for i,text in enumerate(abstracts):
    # Tokenize by sentence, then by lowercase word
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]

    # Process all tokens per quote
    li_tokens_quote = []
    li_tokens_quote_lem = []
    for token in tokens:
        # Remove accents
        t = remove_accents(token)

        # Remove punctuation
        t = str(t).translate(string.punctuation)
        li_tokens_quote.append(t)
        
        # Add token that represents "no lemmatization match"
        li_tokens_quote_lem.append("-") # this token will be removed if a lemmatization match is found below

        # Process each token
        if t not in stops:
            if re.search(RE_VALID, t):
                if len(t) >= MIN_STR_LEN:
                    # Note that the POS (Part Of Speech) is necessary as input to the lemmatizer 
                    # (otherwise it assumes the word is a noun)
                    pos = nltk.pos_tag([t])[0][1][:2]
                    pos2 = 'n'  # set default to noun
                    if pos in DI_POS_TYPES:
                      pos2 = DI_POS_TYPES[pos]
                    
                    stem = stemmer.stem(t)
                    lem = lemmatizer.lemmatize(t, pos=pos2)  # lemmatize with the correct POS
                    
                    if pos in POS_TYPES:
                        li_tokens.append((t, stem, lem, pos))

                        # Remove the "-" token and append the lemmatization match
                        li_tokens_quote_lem = li_tokens_quote_lem[:-1] 
                        li_tokens_quote_lem.append(lem)

    # Build list of token lists from lemmatized tokens
    li_token_lists.append(li_tokens_quote)
    
    # Build list of strings from lemmatized tokens
    str_li_tokens_quote_lem = ' '.join(li_tokens_quote_lem)
    li_lem_strings.append(str_li_tokens_quote_lem)
    
# Build resulting dataframes from lists
df_token_lists = pd.DataFrame(li_token_lists) # (100, 549)

print("Tokenized words in the first 5 entries:")
print(df_token_lists.head(5).to_string())

# Replace None with empty string
for c in df_token_lists:
    if str(df_token_lists[c].dtype) in ('object', 'string_', 'unicode_'):
        df_token_lists[c].fillna(value='', inplace=True)

df_lem_strings = pd.DataFrame(li_lem_strings, columns=['lem quote'])

print()
print("")
print("Tokenized, stemed and lemmatized first 5 entries:")
print(df_lem_strings.head(5).to_string())

## tokeniezed words df = df_token_lists
## tokeniezed entry df = df_lem_strings

Tokenized words in the first 5 entries:
             0            1             2             3             4            5          6         7             8         9         10            11       12    13    14    15    16    17    18    19    20    21    22    23    24    25    26
0  construction       safety         blitz            on          both        sides         of       the        border  safework       nsw          None     None  None  None  None  None  None  None  None  None  None  None  None  None  None  None
1           cbd    apartment  construction          shut          down         over  minefield                      of    safety    issues     illawarra  mercury  None  None  None  None  None  None  None  None  None  None  None  None  None  None
2  construction           of    continuing          care      facility       halted        due        to        safety  concerns  canadian  occupational   safety  None  None  None  None  None  None  None  None  None  None 

In [94]:
#tokeniezed words df = df_token_lists
import numpy as np
df_token_lists['rec_id'] = np.arange(1, df_token_lists.shape[0] + 1)

print(df_token_lists.head())

print (df_token_lists.shape) 

df_token_lists_unpivoted = df_token_lists.melt(id_vars=['rec_id'], var_name='word_order', value_name='tokenized_word').drop(['word_order'], axis=1)

df_token_lists_unpivoted = df_token_lists_unpivoted[df_token_lists_unpivoted['tokenized_word'].str.strip().astype(bool)]

print(df_token_lists_unpivoted.head())
df_token_lists_unpivoted.shape 

              0            1             2             3             4  \
0  construction       safety         blitz            on          both   
1           cbd    apartment  construction          shut          down   
2  construction           of    continuing          care      facility   
3   frightening         drop            in  construction        safety   
4           how  inspections           can       improve  construction   

             5          6         7             8         9  ... 18 19 20 21  \
0        sides         of       the        border  safework  ...               
1         over  minefield                      of    safety  ...               
2       halted        due        to        safety  concerns  ...               
3  inspections    exposed     unite           the     union  ...               
4         site     safety  canadian  occupational    safety  ...               

  22 23 24 25 26 rec_id  
0                     1  
1                     

(118323, 2)

In [106]:
## Reference list to token, stem version, lem version and the POS tag,
df_all_words = pd.DataFrame(li_tokens, columns=['token', 'stem', 'lem', 'pos'])
print (f'df2 shape: {df_all_words.shape}') #(49023, 4)

df2 shape: (49023, 4)


In [96]:
# Add counts by pos from all words with POS, 'NN' is also the default value when POS not found
print("Group by lemmatized words, add count and sort:")
df_all_words['counts'] = df_all_words.groupby(['lem'])['lem'].transform('count')
df_all_words = df_all_words.sort_values(by=['counts', 'lem'], ascending=[False, True]).reset_index()

print("Get just the first row in each lemmatized group")
df_words = df_all_words.groupby('lem').first().sort_values(by='counts', ascending=False).reset_index()
print("Top 10 lemmatized words within all entries:")
print(df_words.head(10))
print (f'All lem word with POS data size : {df_words.shape}.') 

Group by lemmatized words, add count and sort:
Get just the first row in each lemmatized group
Top 10 lemmatized words within all entries:
            lem  index         token          stem pos  counts
0  safetyhealth     85  safetyhealth  safetyhealth  NN     900
1         start    296       started         start  VB     300
2          fine    253         fines          fine  NN     200
3         death     56        deaths         death  NN     200
4    technology    117  technologies     technolog  NN     200
5          halt      8        halted          halt  VB     200
6        boston    125        boston        boston  NN     200
7    contractor     87   contractors    contractor  NN     200
8       satnews    322       satnews        satnew  NN     200
9          fall    121       falling          fall  VB     200
(506, 6)


In [97]:
print('POS tags within all abstracts')
df_words.groupby(['pos']).size().sort_values(ascending=False).reset_index(name='count')
## has 4 types of POS across all words

POS tags within all abstracts


,pos,count
0,NN,415
1,VB,50
2,JJ,34
3,RB,7


In [98]:
## Create reference list with raw entry record number 'rec_id','NN' is also the default value when POS not found

### drop duplucate tokens
df_all_words2 = df_all_words.drop_duplicates()
print (f'Removed duplicates shape: {df_all_words2.shape}') 
df_all_words2.head()

## join with df_token_lists by token to get the rec_id which is the record id in the raw data
df_all_token_words = df_token_lists_unpivoted.merge(df_all_words2, how = 'left', left_on='tokenized_word',right_on ='token').drop(['token'], axis=1)

print (f'All token words list shape: {df_all_token_words.shape}')
df_all_token_words.head() 

Removed duplicates shape: (49023, 6)
All token words list shape: (5732963, 7)


,rec_id,tokenized_word,index,stem,lem,pos,counts
0,1,construction,NaN,NaN,NaN,NaN,NaN
1,2,cbd,2.0,cbd,cbd,NN,100.0
2,2,cbd,490.0,cbd,cbd,NN,100.0
3,2,cbd,978.0,cbd,cbd,NN,100.0
4,2,cbd,1466.0,cbd,cbd,NN,100.0


In [99]:
## count words by each entries to see if can summaries the entry a bit more 
df_all_token_words_entry = df_all_token_words.groupby(['rec_id','lem','pos']).size().sort_values(ascending=False).reset_index(name='count')

In [105]:
df_all_token_words_entry

,rec_id,lem,pos,count
0,8243,safetyhealth,NN,1800
1,553,safetyhealth,NN,1800
2,7562,safetyhealth,NN,1800
3,2470,safetyhealth,NN,1800
4,3154,safetyhealth,NN,1800
...,...,...,...,...
48089,3264,constructconnectcom,NN,2
48090,3264,commercial,JJ,2
48091,2880,constructconnectcom,NN,2
48092,2880,commercial,JJ,2


In [100]:
## words within first entry, not able to summaries by the highest number of each POS, as there are many 'NN' with the same count
print('first entry')
df_all_token_words_entry_1 = df_all_token_words_entry[df_all_token_words_entry['rec_id']==1].sort_values('count',ascending=False)
df_all_token_words_entry_1

first entry


,rec_id,lem,pos,count
3721,1,border,NN,100
8133,1,side,NN,100


In [101]:
## get the lem with latest count of the pos, only 4 words, not able to summaries the contains
df_all_token_words_entry_1.loc[df_all_token_words_entry_1.groupby(['rec_id','pos'])['count'].idxmax()]

,rec_id,lem,pos,count
3721,1,border,NN,100


In [102]:
#tokeniezed entry df = df_lem_strings
df_lem_strings['rec_id'] = np.arange(1, df_lem_strings.shape[0] + 1)
df_lem_strings.rename(columns={ df_lem_strings.columns[0]: 'lemmatized entry'}, inplace=True)

print(df_lem_strings.head())
df_lem_strings.shape

                                    lemmatized entry  rec_id
0                      - - - - - side - - border - -       1
1  cbd - - shut - - minefield - - - - illawarra m...       2
2                  - - - care - halt due - - - - - -       3
3         frighten drop - - - - expose unite - union       4
4                                - - - - - - - - - -       5


(9600, 2)

In [104]:
print(f'Original data size: {df.shape}')
print(f'lem_string data size: {df_lem_strings.shape}')
print(f'tokenised word data size: {df_all_token_words.shape}')

Original data size: (9600, 6)
lem_string data size: (9600, 2)
tokenised word data size: (5732963, 7)


In [103]:
# output 2 - raw data with tokenize words for PBI word counts visual, and lemeatized entry for PBI word cloud, all join with rec_id
from functools import reduce

dfs = [df,df_lem_strings,df_all_token_words]

df_merged2 = reduce(lambda  left,right: pd.merge(left,right,on=['rec_id'],
                                            how='outer'), dfs).fillna('')

df_merged2.to_csv(raw_name+'_df_merge_list.csv', index=True)
print(f'The combinded file data size: {df_merged2.shape}.') ##(14439, 9)
df_merged2.head()

(5732963, 13)


,Unnamed: 0,title,source,published_date,abstract,rec_id,lemmatized entry,tokenized_word,index,stem,lem,pos,counts
0,1,Construction safety blitz on both sides of the...,"{'href': 'https://www.safework.nsw.gov.au', 't...",2022-08-16 17:00:00+10:00,Construction safety blitz on both sides of the...,1,- - - - - side - - border - -,construction,,,,,
1,1,Construction safety blitz on both sides of the...,"{'href': 'https://www.safework.nsw.gov.au', 't...",2022-08-16 17:00:00+10:00,Construction safety blitz on both sides of the...,1,- - - - - side - - border - -,safety,,,,,
2,1,Construction safety blitz on both sides of the...,"{'href': 'https://www.safework.nsw.gov.au', 't...",2022-08-16 17:00:00+10:00,Construction safety blitz on both sides of the...,1,- - - - - side - - border - -,blitz,,,,,
3,1,Construction safety blitz on both sides of the...,"{'href': 'https://www.safework.nsw.gov.au', 't...",2022-08-16 17:00:00+10:00,Construction safety blitz on both sides of the...,1,- - - - - side - - border - -,on,,,,,
4,1,Construction safety blitz on both sides of the...,"{'href': 'https://www.safework.nsw.gov.au', 't...",2022-08-16 17:00:00+10:00,Construction safety blitz on both sides of the...,1,- - - - - side - - border - -,both,,,,,
